# 3.7 Neural Networks TensorFlow 2

Here we provide a simple example of Neural Network using TensorFlow 2 with MNIST dataset.

Source:

https://www.tensorflow.org/tutorials/quickstart/beginner

In [ ]:
import tensorflow as tf

The cell below represents the importation of the tensorflow that will be used below. As we keep looking at the code below we also see the conversion of the data from being interegers to floating point numbers 

## 1. Load and prepare the MNIST dataset

Reference:

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is considered another classic dataset in machine learning area. The images are handwritings of English letters. Researchers are using it as a training set to build neural network models that would recognize human handwritings.

The dataset we import from tf.keras is not the original image set, but numerical matrix that transfromed from the images via certain kernels.

*tf.keras.datasets.mnist.load_data()* will return tuples of arrays:

- x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.

- y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.

- x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.

- y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.


In [ ]:
# load the dataset
mnist = tf.keras.datasets.mnist

# convert the samples from integers to floating-point numbers
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


## 2. Build the `tf.keras.Sequential` model by stacking layers. 

Choose an optimizer and loss function for training:

In the cell below there are a few things that are happening, by utilizing numpy for each example the model returns as mentioned below a vector of "logits" or "log-odds" scores for each class. 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3667618 , -0.6238894 , -0.15965405, -0.1678976 ,  0.03871686,
         0.04772881,  0.254204  , -0.33947173, -0.51150054,  0.674971  ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.07233386, 0.05593358, 0.08897898, 0.08824849, 0.10850226,
        0.10948451, 0.13459338, 0.07433505, 0.06258676, 0.20500313]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.2119722

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2996 - accuracy: 0.9133
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1440 - accuracy: 0.9579
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1096 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0901 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0758 - accuracy: 0.9772


## 3. Evaluate the Model

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

Lastly in the main output we see the probability model being exrpessed below in the following array. 

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0799 - accuracy: 0.9766


[0.07987961173057556, 0.9765999913215637]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.9335106e-08, 3.2755800e-09, 1.4847180e-05, 3.2361271e-04,
        5.0750714e-12, 2.9055312e-07, 2.1230453e-13, 9.9965572e-01,
        2.1388543e-07, 5.2338910e-06],
       [1.7507455e-10, 3.9475498e-04, 9.9952865e-01, 7.5630705e-05,
        1.0684393e-13, 6.9549725e-07, 3.3074306e-09, 1.8108823e-12,
        1.9827984e-07, 1.8552683e-13],
       [2.0753700e-06, 9.9811244e-01, 5.3071865e-04, 3.9471797e-05,
        1.9818877e-05, 6.8706790e-06, 3.9716175e-05, 1.1789318e-03,
        6.9543516e-05, 3.6462612e-07],
       [9.9996519e-01, 1.3697474e-11, 1.2248165e-05, 1.2425103e-08,
        3.1461753e-09, 3.8858602e-07, 1.9024083e-05, 2.1883035e-07,
        1.4644231e-08, 2.9214386e-06],
       [3.0520250e-06, 1.5967791e-08, 2.0994905e-05, 6.9902399e-08,
        9.6960104e-01, 2.2239471e-06, 4.3445856e-05, 1.2548335e-04,
        4.8438037e-06, 3.0198848e-02]], dtype=float32)>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.